In [1]:
import re
from collections import defaultdict
import itertools

In [12]:
# def get_layer_inputs(layer_inputs, state_names):
#     actual_layer_inputs = []
#     for layer_input in layer_inputs:
#         actual_inputs = find_matches(layer_input, state_names)
#         # print(f"Layer Inputs: {layer_input}, Actual Inputs: {actual_inputs}")
#         if actual_inputs is None:
#             return None
# 
#         actual_layer_inputs.extend(actual_inputs)
# 
#     return actual_layer_inputs

def create_template_pattern_string(templates, layer_input: str):
    template_pattern_string = layer_input
    for template in templates:
        template_pattern_string = template_pattern_string.replace(template, r"(.+)")

    return template_pattern_string

def find_actual_input_names(layer_input, state_names):
    template_pattern = r"{.*?}"
    templates = re.findall(template_pattern, layer_input)
    # print("Templates: ", templates)
    if not templates:
        # There are no templates => the name of the input is equal to layer_input
        if layer_input in state_names:
            return [layer_input], None
        # Input is not available in the state
        return None, None

    pattern = create_template_pattern_string(templates, layer_input)
    template_values = defaultdict(set)
    actual_names = []
    
    for name in state_names:
        # print(f"re.findall({pattern}, {name}): {re.findall(pattern, name)}")
        templates_value = re.findall(pattern, name)
        if templates_value:
            actual_names.append(name)
            
            for value, template in zip(templates_value[0], templates):
                template_values[template].add(value)

    # Input is not available in the state
    if not actual_names:
        return None, None

    return actual_names, template_values

def replace_templates(name, templates, templates_value):
    for template, value in zip(templates, templates_value):
        name = name.replace(template, value)

    return name

def find_actual_output_names(layer_outputs, template_values):
    actual_outputs = []
    template_pattern = r"{.*?}"
    
    for layer_output in layer_outputs:
        templates = re.findall(template_pattern, layer_output)
        if not templates:
            actual_outputs.append(layer_output)
            continue
            
        for templates_value in itertools.product(*[template_values[template] for template in templates]):
            actual_outputs.append(replace_templates(layer_output, templates, templates_value))
    
    return actual_outputs

In [7]:
actual_names, template_values = find_actual_input_names("test_{id}_{version}", ["test_1_v2", "test_2_v2", "test_3_v1"])
find_actual_output_names(["model_{id}_{version}", "id: {id}", "version: {version}"], template_values)

['model_2_v1',
 'model_2_v2',
 'model_3_v1',
 'model_3_v2',
 'model_1_v1',
 'model_1_v2',
 'id: 2',
 'id: 3',
 'id: 1',
 'version: v1',
 'version: v2']

In [22]:
find_actual_input_names("a_{id}", ["a_1", "a_2", "a_3"])
find_actual_input_names("a", ["a_1", "a_2", "a"])
find_actual_output_names(["model_{id}_{version}", "id: {id_v2}", "version: {version}"], template_values)
find_actual_output_names(["id: {id_V2}"], template_values)
# template_values

[]

In [21]:
l = ["a", "b"]
l.extend([1, 2, 3])
l

['a', 'b', 1, 2, 3]

In [58]:
class Node:
    def __init__(self, name, inputs, outputs, actual_inputs=None, actual_outputs=None, predecessors=None, ):
        self.name = name
        self.inputs = inputs
        self.outputs = outputs
        self.actual_inputs = actual_inputs
        self.actual_outputs = actual_outputs
        self.predecessors = predecessors if predecessors is not None else []
        
    def update_actuals(self, state_producer):
        actual_inputs = []
        actual_outputs = []
        predecessors_set = set()
        
        for input_name in self.inputs:
            actual_names, template_values = find_actual_input_names(input_name, state_producer.keys())
            
            print(f"input_name: {input_name}, state_keys: {state_producer.keys()}, actual_names: {actual_names}, template_values: {template_values}")
            
            if actual_names is None:
                self.actual_inputs = None
                self.actual_outputs = None
                self.predecessors = []
                
                return 
            
            actual_inputs.extend(actual_names)
            actual_outputs.extend(find_actual_output_names(self.outputs, template_values))
        
            
            for actual_name in actual_names:
                predecessors_set.update(state_producer[actual_name])
                
        self.actual_inputs = actual_inputs
        self.actual_outputs = actual_outputs
        self.predecessors = list(predecessors_set)

In [23]:
def test_f(x):
    print(x)
    return x

any(map(lambda x: test_f(x) > 1, range(10)))

0
1
2


True

In [3]:
nodes = [
    Node("A", ["x"], ["y"]),
    Node("A", ["y"], ["z"])
]

user_inputs = {"x": 10}

In [9]:
s = set()
s.update([1, 2])
s.update([10, 2])
s

{1, 2, 10}

In [6]:
state_producer = defaultdict(list)  # state[actual_input_name] => list of producers

for _input in user_inputs.keys():
    state_producer[_input] = []

In [7]:
processing = []
ordered = []
quarantined = []

In [9]:
node = nodes.pop()

processing.append(node)
# while processing:
n = processing.pop()
    
process_node(n, ordered, quarantined, processing, state_producer)


'A'

In [10]:
actual_names = get_layer_inputs(n.inputs, state_producer.keys())


['x']

In [28]:
l = [i for i in range(10) if not i % 2]
l.remove([0, 2, 4])
l

ValueError: list.remove(x): x not in list

In [4]:
from typing import List

In [83]:
def is_any_in_input(input_names, other_names):
    return any(map(lambda x: find_actual_input_names(x, other_names)[0] is not None, input_names))

def all_node_predecessor_are_ordered(node:Node, ordered:List[Node]):
    if node.actual_outputs is None: 
        return False
    
    # this is true if none of node.actual_outputs is taken in input by any of the ordered nodes
    for ordered_node in ordered:
        if not is_any_in_input(ordered_node.inputs, node.actual_outputs):
            return False
    return True

def find_node_successor(node, ordered):
    return [ordered_node for ordered_node in ordered if not is_any_in_input(ordered_node.inputs, node.actual_outputs)]

def process_node(node, ordered, quarantined, processing, state_producer):
    node.update_actuals(state_producer)
    
    print(f"Node: {node.name}, Inputs: {node.inputs}, Actual Inputs: {node.actual_inputs}, Outputs: {node.outputs}, Actual Outputs: {node.actual_outputs}")
    print(f"Ordered: {list(map(lambda x: x.name, ordered))}")
    print(f"Quarantined: {list(map(lambda x: x.name, quarantined))}")
    print(f"Processing: {list(map(lambda x: x.name, processing))}")
    print(f"State Producer: {state_producer}")
    
    if node in quarantined:
        raise Exception(f"Cyclical graph detected! Nodes involved: {dict(zip(map(lambda x: x.name, quarantined), quarantined))}")
    
    if all_node_predecessor_are_ordered(node, ordered):
        ordered.append(node)
        
        for actual_name in node.actual_inputs:
            if node not in state_producer[actual_name]:
                state_producer[actual_name].append(node)
        
        successors = find_node_successor(node, ordered)
        
        if successors:
            quarantined.append(node)
            processing.extend(successors) # extend without duplicates
            
            for successor in successors:
                process_node(successor, ordered, quarantined, processing, state_producer)
        
            quarantined.remove(node)
        
        return True
    
    return False
        

In [91]:
nodes = [
    Node("A", ["x"], ["y"]),
    Node("B", ["y"], ["z"])
]

user_inputs = {"x": 10}

state_producer = defaultdict(list)  # state[actual_input_name] => list of producers

for _input in user_inputs.keys():
    state_producer[_input] = []

processing = []
ordered = []
quarantined = []

In [92]:
i = 0
while nodes:
    node = nodes.pop(0)
    # nodes.append(node)
    
    result_processing = process_node(node, ordered, quarantined, processing, state_producer)
    print(f"Result processing node: {node.name} is: {result_processing}")
    if not result_processing:
        nodes.append(node)
    
    i += 1
    if i > 4:
        break

input_name: x, state_keys: dict_keys(['x']), actual_names: ['x'], template_values: None
Node: A, Inputs: ['x'], Actual Inputs: ['x'], Outputs: ['y'], Actual Outputs: ['y']
Ordered: []
Quarantined: []
Processing: []
State Producer: defaultdict(<class 'list'>, {'x': []})
input_name: x, state_keys: dict_keys(['x']), actual_names: ['x'], template_values: None
Node: A, Inputs: ['x'], Actual Inputs: ['x'], Outputs: ['y'], Actual Outputs: ['y']
Ordered: ['A']
Quarantined: ['A']
Processing: ['A']
State Producer: defaultdict(<class 'list'>, {'x': [<__main__.Node object at 0x000002BE8F17A950>]})


Exception: Cyclical graph detected! Nodes involved: {'A': <__main__.Node object at 0x000002BE8F17A950>}